In [3]:
from langchain.llms.bedrock import Bedrock

In [4]:
# invoke the LLM
llm_claude = Bedrock(
  model_id="anthropic.claude-v2",
  model_kwargs={'max_tokens_to_sample': 200},
  credentials_profile_name="default", 
  region_name="us-east-1"
)

In [14]:
from langchain.embeddings import BedrockEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate

In [10]:
# load data
loader = PyPDFLoader("data/constitucion-politica-col.pdf")
documents_col = loader.load() #
print(f"Documentos cargados={len(documents_col)}")

docs = RecursiveCharacterTextSplitter(
    length_function = len,
    is_separator_regex = False,
    chunk_size=100, 
    chunk_overlap=20
).split_documents(documents_col)


Documentos cargados=2


In [11]:
# define the retriever 
vectorstore_faiss_aws = None
try:
    bedrock_embeddings = BedrockEmbeddings(
        credentials_profile_name="default", region_name="us-east-1"
    )
    vectorstore_faiss_aws = FAISS.from_documents(
        documents=docs,
        embedding = bedrock_embeddings
    )
    print(f"vectorstore faiss con aws bedrock: elementos en el indice={vectorstore_faiss_aws.index.ntotal}")

except ValueError as error:
    print(f"Error conectando con el vector store: {error}")
    raise error

vectorstore faiss con aws bedrock: elementos en el indice=59


In [12]:
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT
print(CONDENSE_QUESTION_PROMPT.template)

Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:


In [20]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

memory_chain = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

qa = ConversationalRetrievalChain.from_llm(
    llm=llm_claude, 
    retriever=vectorstore_faiss_aws.as_retriever(), 
    memory=memory_chain,
    condense_question_prompt=CONDENSE_QUESTION_PROMPT,
    verbose=True, 
    chain_type='stuff', # 'refine',
    #max_tokens_limit=300
)

In [21]:
# the LLMChain prompt to get the answer. the ConversationalRetrievalChange does not expose this parameter in the constructor
qa.combine_docs_chain.llm_chain.prompt = PromptTemplate.from_template("""
{context}

Human: Use at maximum 3 sentences to answer the question inside the <q></q> XML tags. 

<q>{question}</q>

Do not use any XML tags in the answer. If the answer is not in the context say "Sorry, I don't know as the answer was not found in the context"

Assistant:""")

In [22]:
qa.run('colombia es un estado social de derecho?')



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Artículo 1 °. Colombia es un Estado social 
de derecho, organizado en forma de República

Colombia, en su vida, honra, bienes, creencias, y 
demás derechos y libertades, y para asegurar el

PREÁMBULO
EL PUEBLO DE COLOMBIA,
en ejercicio de su poder soberano, representa -

cumplimiento de los deberes sociales del Estado y 
de los particulares.

Human: Use at maximum 3 sentences to answer the question inside the <q></q> XML tags. 

<q>colombia es un estado social de derecho?</q>

Do not use any XML tags in the answer. If the answer is not in the context say "Sorry, I don't know as the answer was not found in the context"

Assistant:

> Finished chain.

> Finished chain.


' <q>colombia es un estado social de derecho?</q>\n\nSí, Colombia es un Estado social de derecho, organizado en forma de República unitaria, descentralizada, con autonomía de sus entidades territoriales, democrática, participativa y pluralista.'

In [23]:
qa.run('cuales son los deberes del estado?')



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: colombia es un estado social de derecho?
Assistant:  <q>colombia es un estado social de derecho?</q>

Sí, Colombia es un Estado social de derecho, organizado en forma de República unitaria, descentralizada, con autonomía de sus entidades territoriales, democrática, participativa y pluralista.
Follow Up Input: cuales son los deberes del estado?
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Artículo 1 °. Colombia es un Estado social 
de derecho, organizado en forma de República

cumplimiento de los deberes sociales del Estado y 
de los particulares.

Colombia, en su vida, honra, bienes, creencias, y 
demás derechos y libertades, y para asegurar el

De los Principi

' <q> Los deberes del Estado colombiano como Estado social de derecho son: garantizar la efectividad de los principios, derechos y deberes consagrados en la Constitución; facilitar la participación de todos en las decisiones que los afectan y en la vida económica, política, administrativa y cultural de la Nación; promover las condiciones para que la igualdad sea real y efectiva y adoptar medidas en favor de grupos discriminados o marginados; y proteger a todas las personas residentes en Colombia en su vida, honra, bienes, creencias y demás derechos y libertades. </q>'

In [24]:
qa.run('deben ser protegidas las personas en colombia?')



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: colombia es un estado social de derecho?
Assistant:  <q>colombia es un estado social de derecho?</q>

Sí, Colombia es un Estado social de derecho, organizado en forma de República unitaria, descentralizada, con autonomía de sus entidades territoriales, democrática, participativa y pluralista.
Human: cuales son los deberes del estado?
Assistant:  <q> Los deberes del Estado colombiano como Estado social de derecho son: garantizar la efectividad de los principios, derechos y deberes consagrados en la Constitución; facilitar la participación de todos en las decisiones que los afectan y en la vida económica, política, administrativa y cultural de la Nación; promover las condiciones para que la igualdad sea real y efectiva y adoptar medidas en favor de grupos discrim

' <q> Según el Artículo 2 de la Constitución Política de Colombia, es deber del Estado garantizar la efectividad de los principios, derechos y deberes consagrados en la Constitución. Por lo tanto, el Estado debe proteger a las personas residentes en Colombia en su vida, honra, bienes, creencias y demás derechos y libertades. </q>'